# Lee refined speckle filter evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

import os


from polsarpro.dev.devtools import parse_psp_parameter_string
import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import read_psp_bin, read_C3_psp, open_netcdf_beam
from polsarpro.speckle_filters import refined_lee
from polsarpro.util import S_to_C3
from polsarpro.dev.metrics import summarize_metrics
import shutil

import matplotlib.pyplot as plt

# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/speckle_filter/"

# change to your data paths
# original dataset
input_alos_data = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")

input_test_dir = Path("/data/psp/SAN_FRANCISCO_ALOS1")

# output files from C
output_test_dir = Path("/data/psp/res/ref_lee_c")
if not output_test_dir.exists():
    output_test_dir.mkdir(parents=True)

## Run the C-version on some test data

In [ ]:
# fl1: alpha, beta, delta, lambda
# fl2: lambda
# fl3: alpha
# fl4: entropy 
# fl5: anisotropy
# fl6-9 combinations

input_str= f"""id: {input_test_dir} 
od: {output_test_dir}
iodf: S2C3
nw: 7
nlk: 1
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
errf: /tmp/MemoryAllocError.txt
mask: {input_test_dir}/mask_valid_pixels.bin
"""
result = parse_psp_parameter_string(input_str)
os.system(f"lee_refined_filter.exe {result}")

# CPSP does not create another config file in output dir
shutil.copy(input_test_dir / "config.txt", output_test_dir)

## Load ALOS data and C outputs

In [ ]:
# uncomment to test on S matrix made with SNAP
S = open_netcdf_beam(input_alos_data)
out_c = read_C3_psp(output_test_dir)

## Apply the decomposition

In [ ]:
file_out = "/data/psp/res/test_h_a_alpha.nc"
# netcdf writer cannot overwrite
if os.path.isfile(file_out):
    os.remove(file_out)

# TODO: write to file
with ProgressBar():
    out_py = refined_lee(S_to_C3(S), window_size=7, num_looks=1).compute()
    

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(out_py.m11[::8][500:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(out_c.m11[::8][500:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)